# An example of reading from Kafka using Python consumer

In [0]:
# cluster connection parameters
params = {
    "bootstrap_servers": "put the bootstrap server address here",
    "security_protocol": "SASL_SSL",
    "sasl_mechanism": "PLAIN",
    "sasl_plain_username": "use API key as username",
    "sasl_plain_password": "use API secret as password"
}

In [0]:
MAIN_TOPIC = "input_topic"

In [0]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(**params)

In [0]:
from kafka import TopicPartition

partitions = [
    TopicPartition(MAIN_TOPIC, partition)
    for partition in consumer.partitions_for_topic(MAIN_TOPIC)
]

In [0]:
from kafka import KafkaProducer

producer = KafkaProducer(**params)

In [0]:
consumer.assign(partitions)
consumer.seek(partitions[0], 0)
BATCH_SIZE = 1024
buffer = []
while True:
    try:
        while len(buffer) <= BATCH_SIZE:
            data = consumer.poll(
                timeout_ms=200,
                # take at most 1024 events
                max_records=BATCH_SIZE,
            )[partitions[0]]
            buffer += [float(item.value.decode("utf-8")) for item in data]
        result = sum(buffer[:BATCH_SIZE]) / len(buffer[:BATCH_SIZE])
        buffer = buffer[BATCH_SIZE:]
        producer.send(
            "output_topic",
            str(result).encode("utf-8")
        )
    except KeyError as error:
        # this helps to ignore delays in the input stream
        pass